**1. Student Interface**

In [1]:
import sqlite3
import pandas as pd
from IPython.display import display, clear_output
pd.options.display.html.table_schema = False

import ipywidgets as widgets

# Connect to SQLite DB
def get_connection():
    return sqlite3.connect("study_track.db")

In [2]:
#Create  Table
conn = get_connection()
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS study_logs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    student_name TEXT,
    subject TEXT,
    hours INTEGER,
    remarks TEXT
)
""")

conn.commit()
conn.close()
print("Table ready!")


Table ready!


In [3]:
#Student Log Input
student_name = widgets.Text(description="Name:")
subject = widgets.Text(description="Subject:")
hours = widgets.IntText(description="Hours:")
remarks = widgets.Textarea(description="Remarks:", layout=widgets.Layout(width="50%"))

save_button = widgets.Button(description="Save Log", button_style="success")
fetch_button = widgets.Button(description="Fetch Logs", button_style="info")
output_student = widgets.Output()


In [4]:
#Save Function
def save_log(b):
    with output_student:
        clear_output()
        conn = get_connection()
        cursor = conn.cursor()
        cursor.execute("""
            INSERT INTO study_logs (student_name, subject, hours, remarks)
            VALUES (?, ?, ?, ?)
        """, (student_name.value, subject.value, hours.value, remarks.value))
        conn.commit()
        conn.close()
        print("Log saved successfully!")

save_button.on_click(save_log)


In [5]:
#Fetch Function
def fetch_logs(b):
    with output_student:
        clear_output()
        conn = get_connection()
        df = pd.read_sql_query("SELECT * FROM study_logs", conn)
        conn.close()
        display(df)

fetch_button.on_click(fetch_logs)


In [6]:
#Student Interface
widgets.VBox([
    widgets.HTML(value="<h3>Student Study Log Interface</h3>"),
    student_name,
    subject,
    hours,
    remarks,
    widgets.HBox([save_button, fetch_button]),
    output_student
])


 **2.  Admin Panel**

In [7]:
#Admin Credentials default
ADMIN_USER = "admin@123"
ADMIN_PASS = "12345678"

#Admin Login
admin_user = widgets.Text(description="Username:")
admin_pass = widgets.Password(description="Password:")
login_button = widgets.Button(description="Login", button_style="primary")
login_output = widgets.Output()
admin_panel_output = widgets.Output()


In [8]:
# Admin Login Logic
def verify_admin(b):
    with login_output:
        clear_output()
        if admin_user.value == ADMIN_USER and admin_pass.value == ADMIN_PASS:
            print("Login successful!")
            display(admin_panel())
        else:
            print("Invalid credentials!")

login_button.on_click(verify_admin)


In [9]:
#CRUD operations
#Read
def admin_read():
    conn = get_connection()
    df = pd.read_sql_query("SELECT * FROM study_logs", conn)
    conn.close()
    return df

#Delete entry
def delete_entry(id_value):
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("DELETE FROM study_logs WHERE id=?", (id_value,))
    conn.commit()
    conn.close()

#Update
def update_entry(id_value, field, new_value):
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute(f"UPDATE study_logs SET {field}=? WHERE id=?", (new_value, id_value))
    conn.commit()
    conn.close()


In [10]:
#Admin UI
def admin_panel():
    df = admin_read()

    id_to_edit = widgets.IntText(description="Record ID:")
    field_to_edit = widgets.Dropdown(
        options=["student_name", "subject", "hours", "remarks"],
        description="Field:"
    )
    new_value = widgets.Text(description="New Value:")

    update_btn = widgets.Button(description="Update", button_style="warning")
    delete_btn = widgets.Button(description="Delete", button_style="danger")

    admin_output = widgets.Output()

    def perform_update(b):
        with admin_output:
            clear_output()
            update_entry(id_to_edit.value, field_to_edit.value, new_value.value)
            print("Record updated!")
            display(admin_read())

    def perform_delete(b):
        with admin_output:
            clear_output()
            delete_entry(id_to_edit.value)
            print("Record deleted!")
            display(admin_read())

    update_btn.on_click(perform_update)
    delete_btn.on_click(perform_delete)

    panel = widgets.VBox([
        widgets.HTML("<h3>Admin Panel</h3>"),
        widgets.HTML("<b>All Student Logs</b>"),
        widgets.Output(),
        id_to_edit,
        field_to_edit,
        new_value,
        widgets.HBox([update_btn, delete_btn]),
        admin_output
    ])

    # display table inside the panel
    panel.children[2].clear_output()
    with panel.children[2]:
        display(df)

    return panel

#Admin Interface
widgets.VBox([
    widgets.HTML("<h3>Admin Login</h3>"),
    admin_user,
    admin_pass,
    login_button,
    login_output,
    admin_panel_output
])


In [12]:
#Testing
print("Fetching current data...")
df = admin_read().copy()     # keep a fresh reference
display(df)

Fetching current data...


,id,student_name,subject,hours,remarks
0,1,Shweta,maths,1,
1,2,Priyanka,Physics,2,
2,3,Madhuri,History,1,
